In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
from scipy.optimize import curve_fit
from scipy.stats import chi2
from scipy.interpolate import interp1d
import os
import sys
import argparse
from utils_paths import path_baofit, path_template
from utils_baofit import wtheta_model, bao_fit
import itertools

# # Argument parser setup
# parser = argparse.ArgumentParser()
# parser.add_argument('--args.cov_type', help='mocks or cosmolike', default='mocks', type=str)
# parser.add_argument('--args.cosmology_template', help='mice, planck or lognormal_Y6BAO', default='mice', type=str)
# parser.add_argument('--args.cosmology_covariance', help='mice, planck or lognormal_Y6BAO', default='mice', type=str)
# parser.add_argument('--args.diag_only', help='y or n', default='n', type=str)
# parser.add_argument('--args.remove_crosscov', help='y or n', default='n', type=str)
# parser.add_argument('--args.delta_theta', help='Values: 0.05, 0.1, 0.15 or 0.2', default=0.2, type=float)
# parser.add_argument('--args.theta_min', help='min theta to do the BAO fit', default=0.5, type=float)
# parser.add_argument('--args.theta_max', help='max theta to do the BAO fit', default=5, type=float)
# parser.add_argument('--args.n_broadband', help='1, 2, 3, 4 or 5', default=3, type=int)
# parser.add_argument('--args.bins_removed', help='bins removed when doing the BAO fit', default='None', type=str)
# parser.add_argument('--args.nz_flag', help='original or modified (shifted+stretched)', default='fid', type=str)
# parser.add_argument('--args.include_wiggles', help='y or n', default='y', type=str)
# parser.add_argument('--args.weight_type', help='Only allowed if fit_data=1. 0 for no weights, 1 for systematic weights applied', default=1, type=int)

# args = parser.parse_args()

# Arguments
class Args:
    def __init__(self):
        self.dataset = "DESY6"
        self.cov_type = "cosmolike_data"
        self.cosmology_template = "planck"
        self.cosmology_covariance = "planck"
        self.diag_only = "n"
        self.remove_crosscov = "n"
        self.delta_theta = 0.2
        self.theta_min = 0.5
        self.theta_max = 5
        self.n_broadband = 3
        self.bins_removed = '0145'
        self.nz_flag = "fid"
        self.include_wiggles = "y"
        self.weight_type = 1
args = Args()
args.include_wiggles = '' if args.include_wiggles == 'y' else '_nowiggles'

# Limits for the fits
alpha_min = 0.8
alpha_max = 1.2

# Handling bins_removed
def generate_bin_mappings():
    bin_mappings = {
        'None': [],
    }
    for r in range(1, 7):
        for combo in itertools.combinations(range(7), r):
            key = ''.join(map(str, combo))
            bin_mappings[key] = list(combo)
    return bin_mappings

# Generate the mappings
bin_mappings = generate_bin_mappings()
args.bins_removed = bin_mappings.get(args.bins_removed, args.bins_removed)

# Galaxy bias dictionary
galaxy_bias = {0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1}

# Redshift distributions
from utils import redshift_distributions
nz_instance = redshift_distributions(args.nz_flag)

# Path for saving the BAO fits
path_baofit_instance = path_baofit(
    include_wiggles=args.include_wiggles, dataset=args.dataset, weight_type=args.weight_type, 
    nz_flag=args.nz_flag, cov_type=args.cov_type, cosmology_template=args.cosmology_template, 
    cosmology_covariance=args.cosmology_covariance, delta_theta=args.delta_theta, 
    theta_min=args.theta_min, theta_max=args.theta_max, n_broadband=args.n_broadband, 
    bins_removed=args.bins_removed, verbose=True
)
path_baofit = path_baofit_instance()

# Path for loading the templates
path_template = path_template(include_wiggles=args.include_wiggles, nz_flag=args.nz_flag, cosmology_template=args.cosmology_template)()

# # Check if the results already exist
# if os.path.exists(f"{path_baofit}/fit_results.txt"):
#     print("This one already done, moving to the next one!")
#     sys.exit()

# Create directory if it doesn't exist
os.makedirs(path_baofit, exist_ok=True)

# 1. Data and theoretical wtheta
indices_theta_allbins = {}

theta_wtheta_data = {}
wtheta_data = {}

theta_wtheta_th = {}
wtheta_th = {}
wtheta_bb_th = {}
wtheta_bf_th = {}
wtheta_ff_th = {}

for bin_z in range(nz_instance.nbins):
    if args.dataset == 'DESY6':
        filename_wtheta = f'wtheta_data_Yband/wtheta_data_bin{bin_z}_DeltaTheta{args.delta_theta}_weights{args.weight_type}_fstar.txt'
    theta, wtheta = np.loadtxt(filename_wtheta).T
    
    indices_theta_individualbin = np.where((theta > args.theta_min * np.pi / 180) & (theta < args.theta_max * np.pi / 180))[0]

    theta_wtheta_data[bin_z] = theta[indices_theta_individualbin]
    if bin_z in args.bins_removed:
        wtheta_data[bin_z] = np.zeros(len(indices_theta_individualbin))
    else:
        wtheta_data[bin_z] = wtheta[indices_theta_individualbin]

    theta_wtheta_th[bin_z] = np.loadtxt(f'{path_template}/wtheta_bb_bin{bin_z}.txt')[:, 0]
    wtheta_bb_th[bin_z] = np.loadtxt(f'{path_template}/wtheta_bb_bin{bin_z}.txt')[:, 1]
    wtheta_bf_th[bin_z] = np.loadtxt(f'{path_template}/wtheta_bf_bin{bin_z}.txt')[:, 1]
    wtheta_ff_th[bin_z] = np.loadtxt(f'{path_template}/wtheta_ff_bin{bin_z}.txt')[:, 1]

    wtheta_th[bin_z] = (
        galaxy_bias[bin_z]**2 * wtheta_bb_th[bin_z] +
        galaxy_bias[bin_z] * wtheta_bf_th[bin_z] +
        wtheta_ff_th[bin_z]
    )

    indices_theta_allbins[bin_z] = indices_theta_individualbin + bin_z * len(theta)

indices_theta_allbins_concatenated = np.concatenate([indices_theta_allbins[i] for i in range(nz_instance.nbins)])

# Interpolate the theoretical wtheta to use it as the template
wtheta_th_interp = {}

for bin_z in range(nz_instance.nbins):
    wtheta_th_interp[bin_z] = interp1d(theta_wtheta_th[bin_z], wtheta_th[bin_z], kind='cubic')

# Concatenate the datavector
theta_wtheta_data_concatenated = np.concatenate([theta_wtheta_data[i] for i in range(nz_instance.nbins)])

wtheta_data_concatenated = np.concatenate([wtheta_data[bin_z] for bin_z in range(nz_instance.nbins)])

# Check if all values in the dictionary are the same
if all(np.array_equal(v, list(theta_wtheta_data.values())[0]) for v in theta_wtheta_data.values()):
    theta_data = list(theta_wtheta_data.values())[0]
    print("All theta values are the same. Using the first one as an array.")
else:
    print("Theta values are different. Something seems to be wrong.")
    sys.exit()

# 2. Covariance matrix
if args.cov_type == 'cosmolike_data':
    if args.cosmology_covariance == 'mice':
        if args.delta_theta != 0.2:
            print(f"No mice cosmolike covariance matrix for delta_theta={args.delta_theta}")
            sys.exit()
        cov = np.loadtxt(
            f"cov_cosmolike/cov_Y6bao_data_DeltaTheta{str(args.delta_theta).replace('.', 'p')}_mask_g_mice.txt"
        )
    elif args.cosmology_covariance == 'planck':
        cov = np.loadtxt(
            f"cov_cosmolike/cov_Y6bao_data_DeltaTheta{str(args.delta_theta).replace('.', 'p')}_mask_g_planck.txt"
        )
    theta_cov = np.loadtxt(f"cov_cosmolike/delta_theta_{args.delta_theta}_binning.txt")[:, 2] * np.pi / 180

# Extend theta covariance across all bins
theta_cov_concatenated = np.concatenate([theta_cov] * nz_instance.nbins)

# Verify consistency between theta from covariance and data
print('This should be 0:', abs(theta_wtheta_data_concatenated - theta_cov_concatenated[indices_theta_allbins_concatenated]).max())
if abs(theta_wtheta_data_concatenated - theta_cov_concatenated[indices_theta_allbins_concatenated]).max() > 1e-5:
    print('The covariance matrix and the wtheta of the mocks do not have the same theta')
    sys.exit()
    
# Adjust covariance matrix for removed bins
cov_adjusted = np.zeros_like(cov)
for bin_z1 in range(nz_instance.nbins):
    for bin_z2 in range(nz_instance.nbins):
        slice_1 = slice(bin_z1 * len(theta_cov), (bin_z1 + 1) * len(theta_cov))
        slice_2 = slice(bin_z2 * len(theta_cov), (bin_z2 + 1) * len(theta_cov))
        if bin_z1 == bin_z2 or (bin_z1 not in args.bins_removed and bin_z2 not in args.bins_removed):
            cov_adjusted[slice_1, slice_2] = cov[slice_1, slice_2]
cov = cov_adjusted

# Remove cross-covariances
if args.remove_crosscov == 'y':
    cov_adjusted = np.zeros_like(cov)
    for bin_z in range(nz_instance.nbins):
        slice_ = slice(bin_z * len(theta_cov), (bin_z + 1) * len(theta_cov))
        cov_adjusted[slice_, slice_] = cov[slice_, slice_]
    cov = cov_adjusted

# Retain only the diagonal
if args.diag_only == 'y':
    cov = np.diag(np.diag(cov))

cov_orig = np.copy(cov)

# Choose the same scale cuts for the covariance matrix that are used for the data
cov = cov_orig[indices_theta_allbins_concatenated[:, None], indices_theta_allbins_concatenated]

# 3. Get the template wtheta
wtheta_model_instance = wtheta_model(alpha_min, alpha_max, args.n_broadband, wtheta_th_interp)

# # 4. First fit attempt. We use curve_fit
# n_params = len(wtheta_model_instance.names_params)
# n_params_true = len(wtheta_model_instance.names_params) - (1 + args.n_broadband) * len(args.bins_removed)
# wtheta_template = wtheta_model_instance.get_wtheta_template()

# popt, pcov = curve_fit(wtheta_template, theta_wtheta_data[0], wtheta_data_concatenated, p0=wtheta_model_instance.p0, bounds=wtheta_model_instance.bounds, sigma=cov, absolute_sigma=False)
# err_params = np.sqrt(np.diag(pcov))

# # Let's compute the chi2 and dof
# wtheta_fit_concatenated = wtheta_template(theta_wtheta_data[0], *popt)
# diff = wtheta_data_concatenated - wtheta_fit_concatenated
# chi_square = diff @ np.linalg.inv(cov) @ diff
# dof = len(wtheta_data_concatenated) - n_params_true
# for bin_z in args.bins_removed:
#     dof -= len(indices_theta_allbins[bin_z])

# print(f'Best-fit alpha = {popt[0]:.4f} ± {np.sqrt(pcov[0, 0]):.4f}')
# print(f'chi2/dof = {chi_square:.4f}/{dof}')

# 5. With our pipeline
bao_fit_instance = bao_fit(wtheta_model_instance, theta_data, wtheta_data, cov, path_baofit)
alpha_best, err_alpha, chi2_best, dof = bao_fit_instance.fit()


Using fid n(z), which has 6 redshift bins
Saving output to: fit_results/DESY6_1/nzfid_covcosmolike_data_plancktemp_planckcov_deltatheta0.2_thetamin0.5_thetamax5_3broadband_binsremoved[0, 1, 4, 5]
Saving output to: wtheta_template/nz_fid/wtheta_planck
All theta values are the same. Using the first one as an array.
This should be 0: 2.7755575615628914e-17
Best-fit alpha = 0.9853 ± 0.0428
chi2/dof = 36.3383/35
